In [ ]:
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())

### Test Local LangGraph Agent

In [ ]:
from langgraph_sdk import get_client

url = "https://leetmock001-76a7d6889223553c93a96358909dd6e3.default.us.langgraph.app"
client = get_client(url=url)

# Using the graph deployed with the name "agent"
assistant_id = "template"

# create thread
thread = await client.threads.create()

print(thread)


In [3]:
openai_assistant = await client.assistants.create(graph_id=assistant_id)

In [ ]:
openai_assistant

In [34]:
import time
from agent_graph.template.graph import create_graph


graph = create_graph()
async def print_time():
    start_t = time.time()
    async for chunk in client.runs.stream(
        thread["thread_id"],
        assistant_id,
        input={"messages": ["HI"], "event": "user_message"},
        stream_mode=["updates"],
        multitask_strategy="interrupt",
    ):
        # tags = chunk.data.get("tags", [])
        # if "chatbot" not in tags:
        #     continue

        # event = chunk.data.get("event", None)
        # if event != "on_chat_model_stream":
        #     continue

        # content = chunk.data.get("data", {}).get("chunk", {}).get("content", "")
        return time.time() - start_t
        print("-" * 100)
        print(chunk)
        print("-" * 100)

async def print_time_local():
    start_t = time.time()
    async for chunk in graph.astream_events(
        input={"messages": ["HI"], "trigger": True},
        version="v2",
    ):
        return time.time() - start_t

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

times = []
for _ in range(200):
    t = await print_time_local()
    times.append(t)
    print(t)

In [ ]:
# make interval to be 0.01
import numpy as np

# Determine the range of the data
min_time = min(times)
max_time = max(times)

# Create bin edges with 0.01 intervals
# plt.xticks(np.arange(min_time, max_time + 0.2, 0.2))

plt.hist(times)
plt.xlabel("Time taken (s)")
plt.ylabel("Frequency")
plt.title("Distribution of time taken to respond")
plt.show()

In [ ]:
sum(times) / len(times)

In [1]:
from agent_graph.code_mock_staged_v1.graph import create_compiled_graph, create_graph
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())

# graph = create_graph().compile()
graph = create_compiled_graph()
config = { "configurable": {"thread_id": "1" } }

In [3]:
import asyncio

a = graph.ainvoke({"messages": ["No!"]}, config=config)
b = graph.ainvoke({"messages": ["Hi"]}, config=config)

In [4]:
graph.get_state(config=config).values

{}

In [3]:
async for chunk in graph.astream(
    input={
        "messages": [],
        "event": "user_message"
        # "trigger": True,
    },
    config=config,
    stream_mode=["updates", "debug", "custom"],
):
    # await graph.aupdate_state(config=config, values={ "messages": ["hi"]}, as_node="init_state")
    # state = await graph.aget_state(config=config)
    # print(state)
    print(chunk)

# async for chunk in graph.astream(
#     input={
#         "messages": ["Hi, my name is Brian. I am a software engineer working at Meta. My background is in computer science and I have a passion for building scalable and efficient systems. My previous projects are in building scalable and efficient systems, as well as building scalable and efficient systems. My goal is to build scalable and efficient systems."],
#         # "event": "user_message"
#         "trigger": True
#     },
#     config=config,
#     stream_mode=["updates"],
# ):
#     print(chunk)
# list(graph.get_state_history(config=config))

('debug', {'type': 'checkpoint', 'timestamp': '2024-11-04T09:04:02.982465+00:00', 'step': 3, 'payload': {'config': {'tags': [], 'metadata': ChainMap({'thread_id': '1'}), 'callbacks': None, 'recursion_limit': 25, 'configurable': {'checkpoint_ns': '', 'thread_id': '1', '__pregel_stream_writer': <function Pregel.astream.<locals>.<lambda> at 0x12cab5bc0>, 'checkpoint_id': '1ef9a8bb-cc35-66f8-8003-a00a9a320b16'}}, 'parent_config': {'tags': [], 'metadata': ChainMap({'thread_id': '1'}), 'callbacks': None, 'recursion_limit': 25, 'configurable': {'checkpoint_ns': '', 'thread_id': '1', '__pregel_stream_writer': <function Pregel.astream.<locals>.<lambda> at 0x12cab5bc0>, 'checkpoint_id': '1ef9a8bb-4d81-6ae6-8002-26289858362d'}}, 'values': {'messages': [HumanMessage(content='(User just joined the call, please welcome and introduce yourself:)', id='8c16bddc-09d3-4eaa-86e5-0b7774f89189')], 'initialized': True, 'trigger': True, 'steps': {'intro': [Step(name='introduce_self', description='Concisely in

In [ ]:
r = await graph.ainvoke({ "messages": ["Hi, my name is Brian. I am a software engineer working at Meta. My background is in computer science and I have a passion for building scalable and efficient systems. My previous projects are in building scalable and efficient systems, as well as building scalable and efficient systems. My goal is to build scalable and efficient systems."], "trigger": True }, config=config)
r

In [ ]:
from langchain_core.load.load import load, loads
from langchain_core.load.dump import dumpd, dumps
from langgraph.types import StateSnapshot

state = graph.get_state(config=config)
state

In [ ]:
dumped_state = dumps(state)
dumped_state

In [ ]:
loaded_state = loads(dumped_state, valid_namespaces=["agent_graph"])
loaded_state_snapshot = StateSnapshot(*loaded_state)
loaded_state_snapshot

In [ ]:
loaded_state_snapshot == state, dumpd(loaded_state_snapshot) == dumpd(state)

In [ ]:
loaded_state_snapshot.values == state.values

In [60]:
history = list(graph.get_state_history(config=config))

In [ ]:
dumped_history = dumps(history)
dumped_history

In [ ]:
loaded_history = loads(dumped_history, valid_namespaces=["agent_graph"])
loaded_history_snapshot = [StateSnapshot(*h) for h in loaded_history]
loaded_history_snapshot

In [ ]:
loaded_history_snapshot == history, dumpd(loaded_history_snapshot) == dumpd(history)

In [ ]:
dumped_state_byte = dumped_state.encode()
dumped_history_byte = dumped_history.encode()
print(f"State: {len(dumped_state_byte) / 1024:.2f} KB")
print(f"State History: {len(dumped_history_byte) / 1024:.2f} KB")

In [ ]:
from langgraph.version import __version__

__version__


In [10]:
from typing import Annotated
from langgraph.graph.state import StateGraph, CompiledStateGraph
from langgraph.graph import add_messages
from pydantic.v1 import BaseModel, Field
from langgraph.checkpoint.memory import MemorySaver

class State(BaseModel):
    messages: Annotated[list[str], add_messages] = Field(default_factory=list)


g = StateGraph(State)
gg = g.compile(checkpointer=MemorySaver())

In [16]:
gg.invoke({"messages": ["hi 123"]}, config={ "configurable": { "thread_id": "1" } })

{'messages': [HumanMessage(content='hi 123', id='1f557f49-2fd7-4144-b9a8-219f9a88decd'),
  HumanMessage(content='hi 123', id='b65fca32-b725-432e-a278-df00016b4cab'),
  HumanMessage(content='hi 123', id='dec091ee-349d-45ce-82ba-6b5972f6ddcc'),
  HumanMessage(content='hi 123', id='76112518-f75b-4a56-a6be-c337ef7d16e0')]}

In [17]:
gg.get_state(config={ "configurable": { "thread_id": "1" } }).values


{'messages': [HumanMessage(content='hi 123', id='1f557f49-2fd7-4144-b9a8-219f9a88decd'),
  HumanMessage(content='hi 123', id='b65fca32-b725-432e-a278-df00016b4cab'),
  HumanMessage(content='hi 123', id='dec091ee-349d-45ce-82ba-6b5972f6ddcc'),
  HumanMessage(content='hi 123', id='76112518-f75b-4a56-a6be-c337ef7d16e0')]}

In [18]:
gg.invoke(State(messages=["hi 123 456"]), config={"configurable": {"thread_id": "1"}})


{'messages': [HumanMessage(content='hi 123', id='1f557f49-2fd7-4144-b9a8-219f9a88decd'),
  HumanMessage(content='hi 123', id='b65fca32-b725-432e-a278-df00016b4cab'),
  HumanMessage(content='hi 123', id='dec091ee-349d-45ce-82ba-6b5972f6ddcc'),
  HumanMessage(content='hi 123', id='76112518-f75b-4a56-a6be-c337ef7d16e0'),
  HumanMessage(content='hi 123 456', id='7618114f-fe78-4fa8-bf45-6dbe34df7d71')]}